In [70]:
%matplotlib inline

import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.automl import H2OAutoML

import matplotlib.pyplot as plt
import numpy as np

In [71]:
# constant random seed 
seed = 3232

# Connect to a pre-existing cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 1 hour 53 mins
H2O cluster timezone:,America/Denver
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.8
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_pivotal_g3qr87
H2O cluster total nodes:,1
H2O cluster free memory:,1.280 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [72]:
from h2o.utils.shared_utils import _locate # private function. used to find files within h2o git project directory.

#df = h2o.import_file(path=_locate("storymanagement.csv"))
df = h2o.import_file(path=_locate("infra_old.csv"))
# df = dfi.na_omit()


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [73]:
df.describe()

Rows:13997
Cols:18




,Id,Title,Labels,Iteration,Iteration Start,Iteration End,Type,Estimate,Current State,Created at,Accepted at,Deadline,Requested By,Description,URL,Owned By,Owned By 2,Owned By 3
type,int,string,enum,int,enum,enum,enum,int,enum,enum,enum,string,enum,string,string,enum,enum,enum
mins,1.0,NaN,,1.0,,,,0.0,,,,NaN,,NaN,NaN,,,
mean,49031560.6875,NaN,,349.669315801,,,,1.01371769384,,,,0.0,,NaN,NaN,,,
maxs,154943249.0,NaN,,646.0,,,,8.0,,,,NaN,,NaN,NaN,,,
sigma,50950125.2707,NaN,,174.572012043,,,,0.97595403346,,,,-0.0,,NaN,NaN,,,
zeros,0,0,,0,,,,1563,,,,0,,0,0,,,
missing,156,106,6030,156,149,151,152,8967,155,588,154,13983,157,5533,156,1056,11230,13516
0,16.0,Get prioritization working again,,1.0,"Sep 26, 2005","Oct 2, 2005",feature,1.0,accepted,"Sep 12, 2006","Sep 26, 2005",,Nathan Wilmes,,https://www.pivotaltracker.com/story/show/16,,,
1,15.0,"Color-code iterations - grey for past, blue for current, green for future",,1.0,"Sep 26, 2005","Oct 2, 2005",feature,1.0,accepted,,"Sep 26, 2005",,Nathan Wilmes,,https://www.pivotaltracker.com/story/show/15,,,
2,4.0,"Save button on ""edit user"" is incorrectly labeled ""edit"".",,1.0,"Sep 26, 2005","Oct 2, 2005",bug,nan,accepted,,"Sep 27, 2005",,Rob Mee,,https://www.pivotaltracker.com/story/show/4,,,


In [74]:
# Does preprocessing on the raw data frame
def preprocess(df):
    df_edited = df
    
    for field in ["Id", "Current State", "Requested By", "URL", "Zendesk ID", "Integration", "Labels",
                  "Owned By", "Owned By 2", "Owned By 3", "Iteration Start", "Iteration End", "Created at", 
                  "Accepted at", "Deadline", "Title", "Description"
                 ]: 
        try:
            df_edited = df_edited.drop(field)
        except:
            pass

    # Convert fields to logical factors
    df_edited['Type'] = df_edited['Type'].asfactor() #drop non-features altogether?
    
    return df_edited


In [75]:
df_factors = preprocess(df)

In [76]:
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

# Break titles into sequence of words
descriptions = tokenize(df["Description"].ascharacter())
titles = tokenize(df["Title"].ascharacter())

x = df["Description"].ascharacter().cbind(df["Title"].ascharacter())
all_text = tokenize(x)
titles.head()
descriptions.head()
all_text.head()

C1
get
prioritization
working
""
color
code
iterations
grey
past
blue


In [77]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

# This takes time to run - left commented out
description_w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
description_w2v_model.train(training_frame=all_text)

# title_w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
# title_w2v_model.train(training_frame=titles)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [78]:
description_w2v_model.find_synonyms("command", count = 5)

OrderedDict([(u'commands', 0.74235999584198),
             (u'storycreate', 0.7106621861457825),
             (u'epicupdate', 0.7018342018127441),
             (u'addstory', 0.6913412809371948),
             (u'addcomment', 0.6774505376815796)])

In [79]:
# title_w2v_model.find_synonyms("count", count=5)

In [80]:
description_vecs = description_w2v_model.transform(descriptions, aggregate_method = "AVERAGE")
title_vecs = description_w2v_model.transform(titles, aggregate_method = "AVERAGE")
title_vecs.names = ["title_" + t for t in title_vecs.names]

title_vecs

title_C1,title_C2,title_C3,title_C4,title_C5,title_C6,title_C7,title_C8,title_C9,title_C10,title_C11,title_C12,title_C13,title_C14,title_C15,title_C16,title_C17,title_C18,title_C19,title_C20,title_C21,title_C22,title_C23,title_C24,title_C25,title_C26,title_C27,title_C28,title_C29,title_C30,title_C31,title_C32,title_C33,title_C34,title_C35,title_C36,title_C37,title_C38,title_C39,title_C40,title_C41,title_C42,title_C43,title_C44,title_C45,title_C46,title_C47,title_C48,title_C49,title_C50,title_C51,title_C52,title_C53,title_C54,title_C55,title_C56,title_C57,title_C58,title_C59,title_C60,title_C61,title_C62,title_C63,title_C64,title_C65,title_C66,title_C67,title_C68,title_C69,title_C70,title_C71,title_C72,title_C73,title_C74,title_C75,title_C76,title_C77,title_C78,title_C79,title_C80,title_C81,title_C82,title_C83,title_C84,title_C85,title_C86,title_C87,title_C88,title_C89,title_C90,title_C91,title_C92,title_C93,title_C94,title_C95,title_C96,title_C97,title_C98,title_C99,title_C100
0.188086,0.127916,-0.129372,0.152486,0.338477,0.00912391,0.157734,-0.356707,-0.0922398,0.170212,0.163349,0.317121,0.0202113,-0.183257,0.0926856,-0.318819,0.172105,-0.0626777,-0.114801,0.12463,0.0384651,-0.117826,0.176206,-0.0650803,0.161323,-0.117465,0.0539929,0.157668,0.0596385,0.0971185,-0.178937,-0.0970565,-0.145844,-0.0570755,0.187783,0.297012,-0.176955,0.229251,-0.331398,0.266274,-0.0340301,0.0715334,-0.0641393,-0.0894931,0.161287,-0.0973709,-0.0367113,-0.279735,-0.00192801,0.0970281,-0.133757,0.39448,0.0348548,0.0876515,0.109434,0.0979144,-0.0726092,0.0465762,-0.150089,0.0617235,0.0620913,-0.125935,-0.119216,-0.212368,-0.0206058,-0.170261,0.123682,-0.114226,-0.0510927,0.207406,-0.228304,-0.328696,0.0200945,0.0634632,0.22674,0.0200425,0.231817,0.106035,-0.0419326,-0.0778826,0.309077,-0.118036,0.0179647,-0.0429497,0.0551529,-0.157458,-0.0680207,0.0836619,-0.0746115,0.0375601,-0.392753,0.0670916,0.0850326,0.0646261,0.317202,0.0143353,-0.0623165,0.358006,0.208896,-0.00748622
0.367214,-0.17953,-0.092695,-0.127673,0.225561,-0.172358,0.0566831,-0.0613419,0.0378748,0.278177,-0.0864713,0.112894,0.354208,-0.136668,-0.105147,-0.0262015,0.116935,0.283287,0.0303628,0.241436,-0.153247,0.326696,-0.0251484,-0.0198338,0.0710071,0.0173082,0.0271255,-0.0859808,-0.366278,-0.00122737,-0.0480669,0.0980849,-0.275612,0.0929639,-0.0822857,-0.0534012,0.116795,0.0278006,-0.0719829,0.10535,0.0786901,0.0137653,-0.141171,-0.141631,-0.0217488,0.0628439,0.00287906,-0.0140959,-0.175107,0.0715026,-0.0727873,0.109603,0.147683,0.135902,0.141889,-0.0378708,-0.0584493,-0.0645161,0.0403031,0.341395,0.0665163,0.0790287,-0.0151153,-0.212877,0.118958,-0.00939733,-0.0423387,-0.0378805,0.0704566,-0.0186536,-0.172085,-0.225194,0.0416812,0.0481183,0.193663,0.089522,-0.239192,0.183827,0.117386,0.0198163,0.331121,-0.0289981,-0.262492,-0.215624,0.0900946,-0.0987128,-0.440045,0.0451287,-0.0302035,0.0927591,-0.152935,-0.102661,0.0226362,0.230832,0.303333,0.0329724,-0.0653111,-0.0271397,0.184695,-0.0659354
0.480815,0.130229,-0.104208,0.204063,0.0150451,0.0246889,-0.0150485,-0.209919,0.0954456,0.158683,0.192029,0.0423531,0.147843,-0.113443,0.0117924,0.016816,0.0718204,0.26121,0.155167,0.258771,0.129193,0.126819,0.197621,0.0441222,0.0723176,-0.112953,-0.208366,-0.0622957,0.0289479,0.0906553,-0.0912643,0.154813,-0.0667661,0.0040231,-0.211172,-0.0334883,-0.0436253,-0.0294151,0.24907,-0.0367554,-0.170907,0.0811243,-0.123655,0.0403053,0.099216,-0.131256,-0.181422,-0.11344,0.250007,0.0181646,0.0295118,-0.0114669,0.0305067,0.0286914,0.193522,0.220647,-0.112283,0.0853194,-0.0658557,0.0154037,-0.0610577,0.0676028,-0.257816,0.0595234,-0.121008,0.0184993,0.138114,-0.0179879,-0.0640675,0.247427,-0.14137,-0.167468,-0.262532,-0.0531595,0.120946,0.0805618,-0.283834,0.153377,0.213355,-0.023394,0.151939,-0.0546763,0.0715877,-0.267246,-0.100548,-0.179754,-0.0403066,0.0420316,0.166459,0.129643,0.0507231,0.0280778,-0.062254,-0.112193,0.478319,0.332348,-0.0428114,-0.100756,0.130444,-0.218748
0.468035,0.185052,0.0545805,0

In [81]:
df_edited = df_factors.cbind(description_vecs)
df_edited_title = df_factors.cbind(description_vecs).cbind(title_vecs)

df_edited.head()

Iteration,Type,Estimate,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
1,feature,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,feature,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,bug,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,bug,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,feature,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,bug,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,bug,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,feature,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,feature,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

In [82]:

train,test,valid = df_edited.split_frame(ratios=[.85, .1], seed = seed)
train_title,test_title,valid_title = df_edited_title.split_frame(ratios=[.85, .1], seed = seed)

In [86]:
train_title.na_omit()
# See that the data is ready
train_title.describe()

Rows:11939
Cols:203




,Iteration,Type,Estimate,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,title_C1,title_C2,title_C3,title_C4,title_C5,title_C6,title_C7,title_C8,title_C9,title_C10,title_C11,title_C12,title_C13,title_C14,title_C15,title_C16,title_C17,title_C18,title_C19,title_C20,title_C21,title_C22,title_C23,title_C24,title_C25,title_C26,title_C27,title_C28,title_C29,title_C30,title_C31,title_C32,title_C33,title_C34,title_C35,title_C36,title_C37,title_C38,title_C39,title_C40,title_C41,title_C42,title_C43,title_C44,title_C45,title_C46,title_C47,title_C48,title_C49,title_C50,title_C51,title_C52,title_C53,title_C54,title_C55,title_C56,title_C57,title_C58,title_C59,title_C60,title_C61,title_C62,title_C63,title_C64,title_C65,title_C66,title_C67,title_C68,title_C69,title_C70,title_C71,title_C72,title_C73,title_C74,title_C75,title_C76,title_C77,title_C78,title_C79,title_C80,title_C81,title_C82,title_C83,title_C84,title_C85,title_C86,title_C87,title_C88,title_C89,title_C90,title_C91,title_C92,title_C93,title_C94,title_C95,title_C96,title_C97,title_C98,title_C99,title_C100
type,int,enum,int,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,1.0,,0.0,-0.613289833069,-0.575609385967,-1.23421835899,-0.526966571808,-0.431601792574,-0.749376773834,-0.507206380367,-0.723794162273,-0.969160795212,-0.544210135937,-0.475648075342,-0.454058796167,-0.465893834829,-0.493187129498,-0.58183401823,-0.675256371498,-0.430630326271,-0.596178770065,-0.485198050737,-0.626087248325,-0.628068983555,-0.61051183939,-0.268756598234,-0.388613969088,-0.535161733627,-0.776922821999,-0.536987841129,-0.493956357241,-0.586057662964,-0.579938769341,-0.534640848637,-0.414664655924,-0.589319646358,-0.724550545216,-0.507375657558,-0.41931733489,-0.72639644146,-0.482137292624,-0.708372712135,-0.430437266827,-0.484024524689,-0.730877757072,-0.580257296562,-0.561119914055,-0.513206124306,-0.424999028444,-0.585996627808,-0.67748850584,-0.820198535919,-0.790945291519,-0.508906006813,-0.412578195333,-0.880422353745,-0.432851165533,-0.398434251547,-0.918032407761,-0.549733877182,-0.507644712925,-0.730715453625,-0.595769405365,-0.595451712608,-0.703657448292,-0.420623838902,-0.650616884232,-0.631075739861,-0.416171878576,-0.414008229971,-0.488304138184,-0.49297362566,-0.393995434046,-0.594390273094,-0.751760303974,-0.513906776905,-0.594174385071,-0.342025011778,-0.410349309444,-0.66924148798,-0.61498606205,-0.677455961704,-0.468721330166,-0.284346610308,-0.568247795105,-0.635349214077,-0.69828492403,-0.506187975407,-0.466769278049,-0.883827865124,-0.436517834663,-0.574395179749,-0.425618290901,-0.731976985931,-0.480838000774,-0.64013004303,-0.36668741703,-0.472652614117,-0.36119312048,-0.519134998322,-0.702114760876,-0.5

In [87]:
predictors = ["Iteration", "Type"] + description_vecs.names
predictors_title = ["Iteration", "Type"] + title_vecs.names

response = "Estimate"

# gbm_embeddings = H2OGradientBoostingEstimator(stopping_metric = "AUC", stopping_tolerance = 0.001,
#                                               stopping_rounds = 5, score_tree_interval = 10,
#                                               model_id = "gbm_embeddings.hex"
#                                              )

In [88]:
# Run GBM
gbm_model = H2OGradientBoostingEstimator(distribution = "gaussian", ntrees=100, learn_rate=.01)
gbm_model.train(x=predictors, y="Estimate", training_frame=train, validation_frame=valid)

# Run GBM for title
gbm_model_title = H2OGradientBoostingEstimator(distribution = "gaussian", ntrees=100, learn_rate=.01)
gbm_model_title.train(x=predictors_title, y="Estimate", training_frame=train_title, validation_frame=valid_title)


gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [89]:
# print("With Embeddings AUC: " + str(round(gbm_model.auc(valid = True), 3)))
gbm_metrics = gbm_model.model_performance(test)
gbm_metrics


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 1.03016507946
RMSE: 1.01497048207
MAE: 0.672908011954
RMSLE: 0.458723991326
Mean Residual Deviance: 1.03016507946


In [90]:
gbm_metrics_title = gbm_model_title.model_performance(test_title)
gbm_metrics_title


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 0.991456531289
RMSE: 0.995719102603
MAE: 0.664238453888
RMSLE: 0.448884376383
Mean Residual Deviance: 0.991456531289


In [91]:
# Run AutoML
x = train.columns
y = "Estimate"
x.remove(y)
aml = H2OAutoML(max_runtime_secs = 30, seed = seed)
aml.train(x = x, y = y,
          training_frame = train,
          validation_frame = valid,
          leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [92]:
# Run AutoML
x = train_title.columns
y = "Estimate"
x.remove(y)
aml_title = H2OAutoML(max_runtime_secs = 30, seed = seed)
aml_title.train(x = x, y = y,
          training_frame = train_title,
          validation_frame = valid_title,
          leaderboard_frame = test_title)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [93]:

lb = aml.leaderboard
aml_leader_metrics = aml.leader.model_performance(test)

aml_leader_metrics


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 1.01825733804
RMSE: 1.0090873788
MAE: 0.686620541462
RMSLE: 0.450027100646
Mean Residual Deviance: 1.01825733804


In [94]:
lb = aml_title.leaderboard
aml_title_leader_metrics = aml_title.leader.model_performance(test_title)

aml_title_leader_metrics


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 1.0217669235
RMSE: 1.01082487282
MAE: 0.692810550313
RMSLE: 0.447906899316
R^2: 0.0415301545124
Mean Residual Deviance: 1.0217669235
Null degrees of freedom: 472
Residual degrees of freedom: 470
Null deviance: 504.623837566
Residual deviance: 483.295754816
AIC: 1360.50115372


In [ ]:
def predict(stories, w2v, gbm):
    
    words = tokenize(stories["Description"].ascharacter())
    description_vec = w2v.transform(words, aggregate_method="AVERAGE")
    
#     summary_words = tokenize(reviews["Summary"].ascharacter())
#     summary_vec = w2v.transform(summary_words, aggregate_method="AVERAGE")
    
    model_data = stories.cbind(description_vec)#.cbind(summary_vec)
    print(aml.leader.predict(model_data))

#Iteration, type, description
stories = h2o.H2OFrame([
    [15, "feature", "Allow account owners and admins to view and delete github integrations. Super admins can view but not delete", "This includes when a user is looking at the list of github integrations, clicking on a github integration should take the user to the edit or view page depending."],
    [289, "feature", "Allow account owners and admins to view and delete github integrations. Super admins can view but not delete", "This includes when a user is looking at the list of github integrations, clicking on a github integration should take the user to the edit or view page depending."],
    [15, "feature", "Store education box preference via App Settings", "Now that we've seen it work locally, let's add the appropriate logic to the AppSettings endpoint and persist it.\nDo not show preference:\n\nLast time\nRemind me again"],
    [289, "feature", "Store education box preference via App Settings","Now that we've seen it work locally, let's add the appropriate logic to the AppSettings endpoint and persist it.\nDo not show preference:\n\nLast time\nRemind me again"],

])

stories2 = h2o.H2OFrame([
    [15, "feature", "Create the omega primary and warehouse Cloud SQL databases via terraform","Now that we've seen it work locally, let's add the appropriate logic to the AppSettings endpoint and persist it.\nDo not show preference:\n\nLast time\nRemind me again"],
    [647, "feature", "Create the omega primary and warehouse Cloud SQL databases via terraform","Now that we've seen it work locally, let's add the appropriate logic to the AppSettings endpoint and persist it.\nDo not show preference:\n\nLast time\nRemind me again"],
    [647, "feature", "As a Tracker Team Member, I can view a Concourse instance managed by our Infra Team","Spin up in the cf-tracker-production-services project with n >= 1 workers.\n\nAuthentication beyond concourse admin user is out of scope.\n\n## Acceptance criteria\nA concourse instance spun up in the `cf-tracker-production-services` project is accessible via the browser.\n\n---Docs added as part of this story:* https://github.com/pivotaltracker/tracker-docs/blob/master/devops/HowTos/credhub-usage.md* https://github.com/pivotaltracker/tracker-docs/blob/master/devops/HowTos/cloud-workstation-login.md* https://github.com/pivotaltracker/tracker/blob/master/nonapp/concourse-deployment/README.md"]

])

stories.col_names=["Iteration", "Type", "Title", "Description"]

print(predict(stories,description_w2v_model, gbm_model))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
